# Extract and align data from Onix, Harp, Sleap, and photometry
## Cohort 1 and 2 working, Cohort 0: onix_digital Clock column is 0, explore why and/or use timestamps instead 

In [ ]:
import numpy as np
from pathlib import Path
import os
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pandas as pd
import harp
import plotly.express as px
from scipy.stats import mode

import importlib
import harp_resources.process
import harp_resources.utils
# Reassign to maintain direct references
from harp_resources import process, utils

from sleap import load_and_process as lp

In [ ]:
cohort0 = False #only read harp data when it exists, not in Cohort0 
cohort2 = False

#Cohort 1 vestibular mismatch, multiple OnixDigital files 
#data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort1/VestibularMismatch_day1/B6J2718-2024-12-12T13-28-14') #multiple onix_digital file

#Cohort 1 vestibular mismatch, with clock accumulation issue marked on google sheet, seems fine though
#data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort1/VestibularMismatch_day1/B6J2719-2024-12-12T13-59-38') #multiple onix_digital file

#Cohort 1 vestibular mismatch
#data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort1/VestibularMismatch_day1/B6J2717-2024-12-12T13-00-21')

#Cohort 1 visual mismatch 
data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort1/Visual_mismatch_day3/B6J2718-2024-12-10T12-57-02') 

#Cohort 1 visual mismatch
#data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort1/Visual_mismatch_day3/B6J2717-2024-12-10T12-17-03')


#Cohort 0 (no OnixHarp in this Cohort)
#data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort0/Cohort0_GCaMP_example/B3M3xx-2024-08-08T10-05-26')
#cohort0 = True


#Cohort 2 N.B. no videodata in this test set 
#cohort2 = True
#data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort2_like_test_data/2025-01-13T15-47-26')

#Cohort 2 longer test YES OnixHarp! 
#N.B. no photometry in this test set (neitjer videos, but yes video_data)
#cohort2 = True
#data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort2_test_longer/2025-02-10T08-18-59')
 
photometry_path = data_path.parent / f"{data_path.name}_processedData" / "photometry"

#h1_datafolder = data_path / 'HarpDataH1' #only if reading separate registers
#h2_datafolder = data_path / 'HarpDataH2' #only if reading separate registers

In [ ]:
#h1 and h2 only needed if timestamps are readed separately and not as all harp_streams
#h1_reader = harp.create_reader('harp_resources/h1-device.yml', epoch=harp.REFERENCE_EPOCH)
#h2_reader = harp.create_reader('harp_resources/h2-device.yml', epoch=harp.REFERENCE_EPOCH)

session_settings_reader = utils.SessionData("SessionSettings")
experiment_events_reader = utils.TimestampedCsvReader("ExperimentEvents", columns=["Event"])
onix_framecount_reader = utils.TimestampedCsvReader("OnixAnalogFrameCount", columns=["Index"])
#photometry_reader = utils.PhotometryReader("Processed_fluorescence")
video_reader1 = utils.Video("VideoData1")
video_reader2 = utils.Video("VideoData2")
onix_digital_reader = utils.OnixDigitalReader("OnixDigital", columns=["Value.Clock", "Value.HubClock", 
                                                                         "Value.DigitalInputs",
                                                                         "Seconds"])
onix_harp_reader = utils.TimestampedCsvReader("OnixHarp", columns=["Clock", "HubClock", "HarpTime"])

In [ ]:
%%time

print ("Loading session settings")
session_settings = utils.load_2(session_settings_reader, data_path) #Andrew's, creates ugly df, but used in further analysis code
print ("Loading experiment events")
experiment_events = utils.load_2(experiment_events_reader, data_path)

print ("Loading processed fluorescence")
photometry_data=pd.read_csv(str(photometry_path)+'/Processed_fluorescence.csv')
print ("Loading processed fluorescence info")
photometry_info=pd.read_csv(str(photometry_path)+'/Info.csv')
print ("Loading processed fluorescence events")
photometry_events=pd.read_csv(str(photometry_path)+'/Events.csv')

if not cohort2:
    print ("Loading video data 1")
    video_data1 = utils.load_2(video_reader1, data_path)
    print ("Loading video data 2")
    video_data2 = utils.load_2(video_reader2, data_path)

# read Onix data 
print ("Loading OnixDigital")
onix_digital = utils.load_2(onix_digital_reader, data_path)
print ("Loading OnixAnalogFrameClock")
onix_analog_framecount = utils.load_2(onix_framecount_reader, data_path)
print ("Loading OnixAnalogClock")
onix_analog_clock = utils.read_OnixAnalogClock(data_path)
print ("Loading OnixAnalogData")
onix_analog_data = utils.read_OnixAnalogData(data_path, channels = [0], binarise=True, method='adaptive', refractory = 300, flip=True, verbose=False) #method adaptive or threshold (which is hard threshold at 120), refractory to avoid multiple detections

#read HARP data
print ("Loading H1 and H2 streams")
harp_streams = utils.load_registers(data_path, dataframe = True) #loads as df, or if False, as dict 

#read syncronising signal between HARP and ONIX
if not cohort0:
    print ("Loading OnixHarp")
    onix_harp = utils.load_2(onix_harp_reader, data_path)
    onix_harp = utils.detect_and_remove_outliers(
    df=onix_harp,
    x_column="HarpTime",
    y_column="Clock",
    verbose=False  # True prints all outliers
    )
    onix_harp["HarpTime"] = onix_harp["HarpTime"] + 1 # known issue with current version of ONIX, harp timestamps lag 1 second
    print ("Warning: HarpTime +1s to account for know issue with ONIX")

print ("Done Loading")

In [ ]:
importlib.reload(harp_resources.utils)
importlib.reload(harp_resources.process)

# Reassign to maintain direct references
from harp_resources import process, utils

In [ ]:
%%time
(
    conversions, 
    photometry_sync_events, 
    harp_to_onix_clock, 
    onix_time_to_photometry, 
    onix_to_harp_timestamp,
    photometry_to_harp_time
) = process.photometry_harp_onix_synchronisation(
    onix_analog_data=onix_analog_data,
    onix_analog_clock=onix_analog_clock,
    onix_analog_framecount=onix_analog_framecount,
    onix_digital=onix_digital,
    onix_harp=onix_harp,
    photometry_events=photometry_events,
    verbose=True
)


In [ ]:
import numpy as np
import pandas as pd

def resample_to_1khz_grid(experiment_events, photometry_data, onix_analog_clock, onix_analog_data, harp_streams):
    """
    Resamples all datasets to a uniform 1 kHz (1 ms resolution) time grid while preserving alignment.
    - Numeric signals are resampled using interpolation.
    - Boolean signals retain original timestamps.

    Parameters:
        experiment_events (DataFrame): Original event timestamps.
        photometry_data (DataFrame): Photometry data (with a "TimeStamp" column in seconds).
        onix_analog_clock (ndarray): ONIX timestamps (nanoseconds).
        onix_analog_data (ndarray): ONIX data.
        harp_streams (DataFrame): HARP-streamed data (both numeric and boolean).

    Returns:
        dict: A dictionary of aligned datasets.
    """

    # Convert photometry timestamps from seconds to datetime
    photometry_data["Datetime"] = pd.to_datetime(photometry_data["TimeStamp"], unit="s", origin="1900-01-01")
    photometry_data = photometry_data.set_index("Datetime").drop(columns=["TimeStamp"])

    # Convert ONIX timestamps from nanoseconds to datetime
    onix_time_index = pd.to_datetime(onix_analog_clock, unit="ns", origin="1900-01-01")

    # Define a uniform 1 kHz time grid spanning the full experiment
    min_time = min(
        experiment_events.index.min(),
        photometry_data.index.min(),
        onix_time_index.min(),
        harp_streams.index.min()
    )
    max_time = max(
        experiment_events.index.max(),
        photometry_data.index.max(),
        onix_time_index.max(),
        harp_streams.index.max()
    )

    # Create the 1 kHz common time grid
    common_time_grid = pd.date_range(start=min_time, end=max_time, freq="1ms")

    # Debugging print to confirm grid size
    print(f"⚡ Resampling to 1 kHz grid: {len(common_time_grid)} time points")

    # Split harp_streams into numeric and boolean columns
    harp_numeric = harp_streams.select_dtypes(exclude=['bool'])
    harp_bool = harp_streams.select_dtypes(include=['bool'])

    # Resample numeric data to the 1 kHz grid
    def resample_numeric(df):
        """Interpolates numeric data to match the 1 kHz time grid."""
        return df.reindex(df.index.union(common_time_grid)).interpolate(method='time').reindex(common_time_grid)

    # Resample event timestamps, photometry, and ONIX signals
    experiment_events_resampled = resample_numeric(experiment_events)
    photometry_data_resampled = resample_numeric(photometry_data)

    # Interpolate ONIX analog data
    onix_analog_data_resampled = pd.DataFrame(
        index=common_time_grid,
        data=np.interp(
            common_time_grid.astype('int64') / 1e9,  # Convert ms timestamps to seconds
            onix_analog_clock / 1e9,  # Convert ONIX timestamps to seconds
            onix_analog_data
        ),
        columns=["onix_analog_data"]
    )

    # Resample numeric columns of harp_streams to the 1 kHz grid
    harp_numeric_resampled = resample_numeric(harp_numeric)

    # Keep boolean columns at original timestamps (no downsampling)
    harp_bool_aligned = harp_bool.reindex(harp_bool.index.union(common_time_grid)).fillna(method='ffill')

    return {
        "experiment_events_resampled": experiment_events_resampled,
        "photometry_data_resampled": photometry_data_resampled,
        "onix_analog_data_resampled": onix_analog_data_resampled,
        "harp_numeric_resampled": harp_numeric_resampled,
        "harp_bool_aligned": harp_bool_aligned
    }



In [ ]:

# Example Usage:
aligned_data = resample_to_1khz_grid(experiment_events, photometry_data, onix_analog_clock, onix_analog_data, harp_streams)

# # Access the resampled datasets:
# experiment_events_resampled = aligned_data["experiment_events_resampled"]
# photometry_data_resampled = aligned_data["photometry_data_resampled"]
# onix_analog_data_resampled = aligned_data["onix_analog_data_resampled"]
# harp_numeric_resampled = aligned_data["harp_numeric_resampled"]
# harp_bool_aligned = aligned_data["harp_bool_aligned"]


In [ ]:
unique_events = experiment_events["Event"].unique()
print(unique_events)

In [ ]:
# ---- Plotting Parameters ----
window_start = -1  # seconds, analysis window to plot and average
window_stop = 5
how_many_to_plot = -1  # -1 plots all or X plots first x halt events 


if "Visual_mismatch" in str(data_path):    
    block_start_event = "DrumWithReverseHalt block started"
    halt_event = "Apply halt: 2s"
    block_end_event = "Block timer elapsed" # Set to "no_end" to scan all events

if "VestibularMismatch" in str(data_path):
    block_start_event = "Sync signal started"
    halt_event = "DrumWithReverseflow block started"
    block_end_event = "no_end"

In [ ]:
%%time
from scipy.stats import mode


def pad_arrays(array_list):
    """Pads a list of 1D NumPy arrays to the same length using NaN padding."""
    max_len = max(map(len, array_list))  # Efficient max length calculation
    padded_array = np.empty((len(array_list), max_len), dtype=np.float64)  # Preallocate array
    padded_array.fill(np.nan)  # Fill with NaNs in one operation

    for i, arr in enumerate(array_list):
        padded_array[i, :len(arr)] = arr  # Vectorized assignment

    return padded_array

# ---- Extract Halt Events Efficiently ----
block_starts = experiment_events.query("Event == @block_start_event").index.to_numpy()

halt_events_list = []
for block_start in block_starts:
    if block_end_event == "no_end":
        block_halts = experiment_events.query("Event == @halt_event and index > @block_start")
    else:
        block_end = experiment_events.query("Event == @block_end_event and index > @block_start").index.min()
        if pd.notna(block_end):
            block_halts = experiment_events.query("Event == @halt_event and index > @block_start and index < @block_end")
        else:
            block_halts = pd.DataFrame()  # No valid end event found
    
    if not block_halts.empty:
        halt_events_list.append(block_halts)

block_halts = pd.concat(halt_events_list) if halt_events_list else pd.DataFrame()

if block_halts.empty:
    raise ValueError(f"⚠️ No [{halt_event}] events found between [{block_start_event}] and [{block_end_event}]. "
                     f"Check if the event names are correct and exist in experiment_events.")

# Convert Halt Times to NumPy for Efficiency
halt_event_times = block_halts.index.to_numpy()

# Adjust how_many_to_plot if it exceeds available events
if how_many_to_plot > len(block_halts):
    print(f"⚠️ Warning: Requested {how_many_to_plot} halts, but only {len(block_halts)} are available. "
          "Adjusting how_many_to_plot accordingly.")
    how_many_to_plot = len(block_halts)

if how_many_to_plot == -1:
    how_many_to_plot = len(block_halts)  # Limit to avoid excessive plots
print(f"Found {len(block_halts)} halt events within valid blocks, plotting {how_many_to_plot}.")

# Define colors
flow_x_color, flow_y_color, photodiode_color = "blue", "orange", "grey"  # Changed photodiode to grey
z_470_color, z_560_color = "green", "red"

# Initialize lists for aligned data
aligned_time = np.linspace(window_start, window_stop, 500)  
flow_x_aligned, flow_y_aligned, photodiode_aligned, z_470_aligned, z_560_aligned = [], [], [], [], []

# ----------------------
# First Plot: Individual Trials
# ----------------------
fig, ax1 = plt.subplots(figsize=(10, 6))

for idx, halt_time in enumerate(block_halts.index[:how_many_to_plot]):
    halt_time_seconds = halt_time.timestamp()
    min_time, max_time = halt_time + pd.DateOffset(seconds=window_start), halt_time + pd.DateOffset(seconds=window_stop)

    # Extract Optical Tracking Data
    optical_x = harp_streams['OpticalTrackingRead0X(46)'].loc[min_time:max_time].dropna()
    optical_y = harp_streams['OpticalTrackingRead0Y(46)'].loc[min_time:max_time].dropna()

    if not optical_x.empty and not optical_y.empty:
        optical_x_rel = (optical_x.index.astype("int64") / 1e9) - halt_time_seconds
        optical_y_rel = (optical_y.index.astype("int64") / 1e9) - halt_time_seconds

        label_x, label_y = "Flow X" if idx == 0 else None, "Flow Y" if idx == 0 else None
        ax1.plot(optical_x_rel, optical_x, color=flow_x_color, alpha=0.3, label=label_x)
        ax1.plot(optical_y_rel, optical_y, color=flow_y_color, alpha=0.3, label=label_y)

        # Restrict aligned_time to the valid range of optical_x_rel
        valid_mask = (aligned_time >= optical_x_rel.min()) & (aligned_time <= optical_x_rel.max())
        aligned_time_valid = aligned_time[valid_mask]

        # Interpolate only within the valid time range
        flow_x_interp = np.interp(aligned_time_valid, optical_x_rel, optical_x, left=np.nan, right=np.nan)
        flow_y_interp = np.interp(aligned_time_valid, optical_y_rel, optical_y, left=np.nan, right=np.nan)

        # Append only the valid interpolated values
        flow_x_aligned.append(flow_x_interp)
        flow_y_aligned.append(flow_y_interp)

ax1.set_xlabel("Relative Time (s)")
ax1.set_ylabel("Tracking Readout")
ax1.set_title("Optical Flow, Photodiode, and Photometry")

# Create separate y-axis for Photodiode
ax3 = ax1.twinx()
ax3.spines.right.set_position(("outward", 60))
ax3.set_ylabel("Photodiode Signal")
ax3.set_ylim([0, 1.2])

for idx, halt_time in enumerate(block_halts.index[:how_many_to_plot]):
    halt_time_seconds = halt_time.timestamp()

    onix_sec_start_time = harp_to_onix_clock(block_halts.iloc[idx]["Seconds"] + window_start)
    onix_sec_stop_time = harp_to_onix_clock(block_halts.iloc[idx]["Seconds"] + window_stop)

    onix_sec_start_index = np.searchsorted(onix_analog_clock, onix_sec_start_time)
    onix_sec_stop_index = np.searchsorted(onix_analog_clock, onix_sec_stop_time)

    onix_time_rel = (onix_to_harp_timestamp(onix_analog_clock[onix_sec_start_index:onix_sec_stop_index])
                     .astype("int64") / 1e9) - halt_time_seconds

    photodiode_signal = onix_analog_data[onix_sec_start_index:onix_sec_stop_index]

    label_photodiode = "Photodiode" if idx == 0 else None
    ax3.plot(onix_time_rel, photodiode_signal, color=photodiode_color, alpha=0.5, label=label_photodiode)

    # Restrict aligned_time to valid range of onix_time_rel
    valid_mask = (aligned_time >= onix_time_rel.min()) & (aligned_time <= onix_time_rel.max())
    aligned_time_valid = aligned_time[valid_mask]

    # Interpolate photodiode on the valid time range
    photodiode_interp = np.interp(aligned_time_valid, onix_time_rel, photodiode_signal, left=np.nan, right=np.nan)
    photodiode_aligned.append(photodiode_interp)
    
# Create second y-axis for photometry
ax2 = ax1.twinx()
ax2.set_ylabel("Fluorescence Signal")

if "TimeStamp" in photometry_data.columns:
    photometry_data = photometry_data.set_index("TimeStamp")

for idx, halt_time in enumerate(block_halts.index[:how_many_to_plot]):
    halt_time_seconds = halt_time.timestamp()

    photometry_sec_start_time = onix_time_to_photometry(harp_to_onix_clock(block_halts.iloc[idx]["Seconds"] + window_start))
    photometry_sec_stop_time = onix_time_to_photometry(harp_to_onix_clock(block_halts.iloc[idx]["Seconds"] + window_stop))

    photometry_sec = photometry_data.loc[photometry_sec_start_time:photometry_sec_stop_time]

    if not photometry_sec.empty:
        photometry_time_rel = (photometry_to_harp_time(photometry_sec.index).astype("int64") / 1e9) - halt_time_seconds

        label_560, label_470 = "z_560" if idx == 0 else None, "z_470" if idx == 0 else None
        ax2.plot(photometry_time_rel, photometry_sec['z_560'], color=z_560_color, alpha=0.3, label=label_560)
        ax2.plot(photometry_time_rel, photometry_sec['z_470'], color=z_470_color, alpha=0.3, label=label_470)
        
        # Restrict aligned_time to the valid range of photometry_time_rel
        valid_mask = (aligned_time >= photometry_time_rel.min()) & (aligned_time <= photometry_time_rel.max())
        aligned_time_valid = aligned_time[valid_mask]

        # Perform interpolation on the adjusted time range
        z_560_interp = np.interp(aligned_time_valid, photometry_time_rel, photometry_sec['z_560'], left=np.nan, right=np.nan)
        z_470_interp = np.interp(aligned_time_valid, photometry_time_rel, photometry_sec['z_470'], left=np.nan, right=np.nan)
        
        # Append only the valid interpolated values
        z_560_aligned.append(z_560_interp)
        z_470_aligned.append(z_470_interp)

ax1.legend(loc="upper left")
ax2.legend(loc="upper right")
ax3.legend(loc="center right")

plt.show()

# ----------------------
# Second Plot: Averages with Error Shading (Proper Axes Labels & No Overlap)
# ----------------------
fig, ax1 = plt.subplots(figsize=(10, 6))

ax1.set_xlabel("Relative Time (s)")
ax1.set_ylabel("Tracking Readout")

# Pad all signal data to ensure uniform shape
flow_x_aligned_padded = pad_arrays(flow_x_aligned)
flow_y_aligned_padded = pad_arrays(flow_y_aligned)
photodiode_aligned_padded = pad_arrays(photodiode_aligned)
z_560_aligned_padded = pad_arrays(z_560_aligned)
z_470_aligned_padded = pad_arrays(z_470_aligned)

# Ensure all arrays have the same length
# Get the number of valid (non-NaN) values per time point across trials
valid_counts_560 = np.sum(~np.isnan(z_560_aligned_padded), axis=0)
valid_counts_470 = np.sum(~np.isnan(z_470_aligned_padded), axis=0)

# Find the last point where at least 80% of trials still have data
threshold = 0.8 * len(z_560_aligned_padded)  # Adjustable threshold (80%)
adaptive_cutoff = np.where(valid_counts_560 >= threshold)[0][-1]  # Last valid index

# Use the smaller of (1) standard min_length, (2) adaptive cutoff
min_length = min(
    aligned_time.shape[0], 
    photodiode_aligned_padded.shape[1], 
    z_560_aligned_padded.shape[1], 
    z_470_aligned_padded.shape[1], 
    flow_x_aligned_padded.shape[1],  
    flow_y_aligned_padded.shape[1],
    adaptive_cutoff  # Ensure we include this index
)

print(f"🔍 Adaptive cutoff applied at index {adaptive_cutoff}, using min_length = {min_length}")

# Compute means after padding
flow_x_mean = np.nanmean(flow_x_aligned_padded, axis=0)
flow_y_mean = np.nanmean(flow_y_aligned_padded, axis=0)
photodiode_mean = np.nanmean(photodiode_aligned_padded, axis=0)
z_560_mean = np.nanmean(z_560_aligned_padded, axis=0)
z_470_mean = np.nanmean(z_470_aligned_padded, axis=0)

# Compute SEM
flow_x_sem = np.nanstd(flow_x_aligned_padded, axis=0) / np.sqrt(np.sum(~np.isnan(flow_x_aligned_padded), axis=0))
flow_y_sem = np.nanstd(flow_y_aligned_padded, axis=0) / np.sqrt(np.sum(~np.isnan(flow_y_aligned_padded), axis=0))
photodiode_sem = np.nanstd(photodiode_aligned_padded, axis=0) / np.sqrt(np.sum(~np.isnan(photodiode_aligned_padded), axis=0))
z_560_sem = np.nanstd(z_560_aligned_padded, axis=0) / np.sqrt(np.sum(~np.isnan(z_560_aligned_padded), axis=0))
z_470_sem = np.nanstd(z_470_aligned_padded, axis=0) / np.sqrt(np.sum(~np.isnan(z_470_aligned_padded), axis=0))

## Truncate all arrays to match min_length
aligned_time = aligned_time[:min_length]
flow_x_mean = flow_x_mean[:min_length]
flow_y_mean = flow_y_mean[:min_length]
photodiode_mean = photodiode_mean[:min_length]
z_560_mean = z_560_mean[:min_length]
z_470_mean = z_470_mean[:min_length]

# Truncate SEM values to match min_length
flow_x_sem = flow_x_sem[:min_length]
flow_y_sem = flow_y_sem[:min_length]
photodiode_sem = photodiode_sem[:min_length]
z_560_sem = z_560_sem[:min_length]
z_470_sem = z_470_sem[:min_length]

# Plot flow means with SEM shading
ax1.plot(aligned_time, flow_x_mean, color=flow_x_color, label="Flow X (Mean)")
ax1.fill_between(aligned_time, flow_x_mean - flow_x_sem, flow_x_mean + flow_x_sem, color=flow_x_color, alpha=0.2)

ax1.plot(aligned_time, flow_y_mean, color=flow_y_color, label="Flow Y (Mean)")
ax1.fill_between(aligned_time, flow_y_mean - flow_y_sem, flow_y_mean + flow_y_sem, color=flow_y_color, alpha=0.2)

# Photodiode Signal
ax3 = ax1.twinx()
ax3.set_ylabel("Photodiode Signal")
ax3.spines.right.set_position(("outward", 60))
ax3.plot(aligned_time, photodiode_mean, color=photodiode_color, label="Photodiode (Mean)")
ax3.fill_between(aligned_time, photodiode_mean - photodiode_sem, photodiode_mean + photodiode_sem, color=photodiode_color, alpha=0.2)

# Photometry Fluorescence
ax2 = ax1.twinx()
ax2.set_ylabel("Fluorescence Signal")
ax2.plot(aligned_time, z_560_mean, color=z_560_color, label="z_560 (Mean)")
ax2.fill_between(aligned_time, z_560_mean - z_560_sem, z_560_mean + z_560_sem, color=z_560_color, alpha=0.2)

ax2.plot(aligned_time, z_470_mean, color=z_470_color, label="z_470 (Mean)")
ax2.fill_between(aligned_time, z_470_mean - z_470_sem, z_470_mean + z_470_sem, color=z_470_color, alpha=0.2)

ax1.legend(loc="upper left")
ax2.legend(loc="upper right")
ax3.legend(loc="center right")

plt.show()


In [ ]:
onix_harp